Vamos explorar os comentarios na ótica de Detrator vs Promotor para identificar valores de NPS\
Para isso vamos usar LLM (gpt-4o-mini) para aproveitar a caracteristica 'ready-to-use' e bem eficiente para essa classificação

Uma alternativa aqui poderia ser utilizar TF-IDF + Regressor, mas para isso dependeriamos de dados anotados,\
Talvez no futuro possamos coletar a nota de NPS junto já servindo como uma label e tornando esse processo mais rapido e barato em novas execuções

In [ ]:
import time
from utils.nlp_classifier import classificar_comentario
import pandas as pd

df = pd.read_csv('data/03_churn_cleaned.csv')

labels = []
for comentario in df["comments"]:
    label = classificar_comentario(comentario)
    print(f"Comentário: {comentario}\n➡ Classificação: {label}\n")
    labels.append(label)
    time.sleep(1) # avoid rate limit

df["classificacao"] = labels
df.to_csv("data/04_nlp_classificados.csv", index=False)

Comentário: O serviço de internet tem sido instável. Estou tendo dificuldades para realizar videochamadas e assistir a filmes online. A velocidade da conexão é muito lenta e cai com frequência.
➡ Classificação: Detrator

Comentário: Estou satisfeita com o serviço. A velocidade da internet é ótima e o suporte técnico sempre foi prestativo. Nunca tive problemas de conexão desde que assinei o serviço.
➡ Classificação: Promotor

Comentário: Tive problemas com a fatura deste mês. O valor cobrado está incorreto e não consigo entrar em contato com o atendimento ao cliente. Já enviei diversos emails e tentei ligar várias vezes, mas não obtive resposta.
➡ Classificação: Detrator

Comentário: Sem reclamações até o momento. O serviço tem atendido às minhas expectativas e não tenho tido problemas com a conexão. As ligações são claras e o atendimento ao cliente é bom.
➡ Classificação: Promotor

Comentário: O canal de esportes não está funcionando corretamente. A imagem fica pixelada e o áudio está 

Em validação manual os valores obtidos parecem bem coerentes

___

Vamos explorar como está nossa nota atual de NPS, considerando as classificações do LLM

Baseado na formula tradicional de NPS:

$NPS = \frac{\text{Promotores}}{\text{Total}} - \frac{\text{Detratores}}{\text{Total}}$

Que varia de -100 a 100
E tem algumas zonas de interesse:
| Faixa de NPS  | Classificação           | Interpretação comum                   |
| ------------- | ----------------------- | ------------------------------------- |
| **90 a 100**  | Zona de excelência      | Lealdade excepcional                  |
| **70 a 89**   | Zona de qualidade       | Altamente recomendável                |
| **50 a 69**   | Zona de sucesso         | Muito bom, mas pode melhorar          |
| **0 a 49**    | Zona de aperfeiçoamento | Ok, mas ainda há espaço para melhorar |
| **-1 a -100** | Zona crítica            | Urgente investigar problemas          |


In [ ]:
import pandas as pd
from utils.nlp_classifier import calcular_nps

df = pd.read_csv('data/04_nlp_classificados.csv')

nps_geral = calcular_nps(df, verbose=True)

Promotores: 48
Detratores: 47
Neutros: 7
NPS: 0.98


O NPS atual descreve .98, praticamente 1 ponto, o que nos coloca na beira da Zona Critica, mas ainda na Zona de Aperfeiçoamento

### Explorando NPS de sub-grupos

Percebo 4 atributos que podem ter relação com a satisfação do cliente:
1. location (já que podemos ter localidades com qualidades diferentes de serviço)
2. service (já que podemos ter serviços com qualidades diferentes)
3. qtd_dependets (já que isso impacta diretamente em como os serviços sao consumidos)
4. price (já que eu posso tanto ter clientes insatisfeitos com os serviços mais basicos,\
          como clientes mais exigentes com os serviços mais caros)

Vamos explorar como é o NPS em cada um dos subgrupos desses atributos

In [20]:
for col in ['location', 'service', 'qtd_dependents', 'price']:
    print(f"\nNPS por '{col}':")
    for value in df[col].unique():
        temp_df = df[df[col]==value]
        nps = calcular_nps(temp_df, verbose=False)
        print(f"    {value}: {nps}")



NPS por 'location':
    SP: -13.79
    RJ: 8.7
    MG: 8.0
    RS: 4.55
    DF: 0.0

NPS por 'service':
    Telefonia Móvel: -8.7
    Internet: 3.23
    Telefonia Fixa: 26.09
    TV a Cabo: -16.0

NPS por 'qtd_dependents':
    2: -4.0
    0: 14.89
    1: -19.23
    3: 0.0

NPS por 'price':
    79.99: 0.0
    109.99: 6.67
    139.99: 60.0
    159.99: 33.33
    99.99: -28.57
    89.99: -53.85
    119.99: 55.56
    149.99: -53.85
    179.99: 20.0
    69.99: -100.0
    129.99: 69.23
    189.99: 100.0


Todos os valores de location, service, qtd_dependets flutuam nas duas faixas mais baixas: Zona Critica e Zona de Aprimoramento.

Tendo uma tensão um pouco maior em SP.\
E um pouco melhor em Telefonia Fixa.

Quanto a hipotese de que maior numero de dependentes impactar negativamente na satisfação do cliente:\
isso não é sustentado pelos valores de NPS nesses subgrupos

Em especial ao olhar para preço temos uma flutuação bem grande que provavelmente se deve ao tamanho da base\
O que faz com que a quantidade de cada classe do NPS seja bem pequena, podemos tornar essa analise mais robusta utilizando faixas de preço\
ao inves do preço em si, caso essa informação pareça relevante mais pra frente

Nos proximos IPYNB vamos explorar ações que podem melhorar esses valores
